In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

os.chdir(r'D:\data485')
events = pd.read_csv("Galapagos Tortoise Movement Ecology Programme_2009-2018.csv",sep=',')

In [2]:
events

,event-id,visible,timestamp,location-long,location-lat,eobs:battery-voltage,eobs:fix-battery-voltage,eobs:horizontal-accuracy-estimate,eobs:key-bin-checksum,eobs:speed-accuracy-estimate,eobs:start-timestamp,eobs:status,eobs:temperature,eobs:type-of-fix,eobs:used-time-to-get-fix,ground-speed,heading,height-above-ellipsoid,manually-marked-outlier,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,33691529,True,2010-09-23 16:00:00.000,NaN,NaN,3686,3598,NaN,2636949985,NaN,2010-09-23 16:00:00.000,D,17,0,150,NaN,NaN,NaN,NaN,gps,Chelonoidis,1392,Sparkey,Galapagos Tortoise Movement Ecology Programme
1,33691530,True,2010-09-23 17:00:49.998,-91.103888,-0.456140,3696,3623,3.07,4135080321,2.32,2010-09-23 17:00:00.000,A,17,3,49,0.17,16.38,1033.3,NaN,gps,Chelonoidis,1392,Sparkey,Galapagos Tortoise Movement Ecology Programme
2,33691531,True,2010-09-23 18:00:32.000,-91.103888,-0.456140,3702,3635,3.84,3798461165,2.57,2010-09-23 18:00:01.000,A,19,3,31,0.33,3.28,1035.9,NaN,gps,Chelonoidis,1392,Sparkey,Galapagos Tortoise Movement Ecology Programme
3,33691532,True,2010-09-23 19:00:29.000,-91.103904,-0.456151,3710,3642,3.84,3942563785,2.56,2010-09-23 19:00:00.000,A,22,3,29,0.12,19.66,1034.9,NaN,gps,Chelonoidis,1392,Sparkey,Galapagos Tortoise Movement Ecology Programme
4,33691533,True,2010-09-23 20:00:55.999,-91.103904,-0.456153,3710,3635,3.07,683273842,2.28,2010-09-23 19:59:59.000,A,22,3,56,0.19,6.55,1038.5,NaN,gps,Chelonoidis,1392,Sparkey,Galapagos Tortoise Movement Ecology Programme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841248,7674762948,True,2018-10-03 10:00:00.000,NaN,NaN,3890,3841,NaN,2190247491,NaN,2018-10-03 10:00:00.000,D,9,0,150,NaN,NaN,NaN,NaN,gps,Chelonoidis porteri,5264,Samuel,Galapagos Tortoise Movement Ecology Programme
1841249,7674762949,True,2018-10-03 11:00:00.000,NaN,NaN,3885,3833,NaN,563059177,NaN,2018-10-03 11:00:00.000,D,9,0,150,NaN,NaN,NaN,NaN,gps,Chelonoidis porteri,5264,Samuel,Galapagos Tortoise Movement Ecology Programme
1841250,7674762950,True,2018-10-03 12:00:00.000,NaN,NaN,3885,3831,NaN,4083794979,NaN,2018-10-03 12:00:00.000,D,9,0,150,NaN,NaN,NaN,NaN,gps,Chelonoidis porteri,5264,Samuel,Galapagos Tortoise Movement Ecology Programme
1841251,7674762951,True,2018-10-03 13:00:00.000,NaN,NaN,3880,3824,NaN,2610093027,NaN,2018-10-03 13:00:00.000,D,9,0,150,NaN,NaN,NaN,NaN,gps,Chelonoidis porteri,5264,Samuel,Galapagos Tortoise Movement Ecology Programme


Now, define a function to compute the distance by longitudes and latitudes.

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers. Use 3956 for miles
    r = 6371
    return c * r

In [4]:
turtles_distance = events

In [5]:
turtles_distance = turtles_distance.sort_values(by=['individual-local-identifier', 'eobs:start-timestamp'])
turtles_distance = turtles_distance[['individual-local-identifier', 'eobs:start-timestamp', 'location-long', 'location-lat']]
turtles_distance['eobs:start-timestamp'] = pd.to_datetime(turtles_distance['eobs:start-timestamp'])

# Shift latitude and longitude to calculate distance between consecutive rows
turtles_distance['prev_lat'] = turtles_distance.groupby('individual-local-identifier')['location-lat'].shift(1)
turtles_distance['prev_lon'] = turtles_distance.groupby('individual-local-identifier')['location-long'].shift(1)

# Calculate distance
turtles_distance['distance_km'] = turtles_distance.apply(lambda row: haversine(row['prev_lat'], row['prev_lon'], row['location-lat'], row['location-long']) 
                             if pd.notnull(row['prev_lat']) else 0, axis=1)

In [6]:
# Extract date from the timestamp
turtles_distance['date'] = turtles_distance['eobs:start-timestamp'].dt.date

# Group by turtle_name and date, and sum the distances
daily_distance = turtles_distance.groupby(['individual-local-identifier', 'date'])['distance_km'].sum().reset_index()

print(daily_distance)

       individual-local-identifier        date  distance_km
0                           Alison  2010-09-17     0.097282
1                           Alison  2010-09-18     0.260619
2                           Alison  2010-09-19     0.205049
3                           Alison  2010-09-20     0.256245
4                           Alison  2010-09-21     0.243576
...                            ...         ...          ...
108536                       Zelfa  2016-06-12     0.123130
108537                       Zelfa  2016-06-13     0.094405
108538                       Zelfa  2016-06-14     0.070772
108539                       Zelfa  2016-06-15     0.171706
108540                       Zelfa  2016-06-16     0.018194

[108541 rows x 3 columns]


In [7]:
daily_distance

,individual-local-identifier,date,distance_km
0,Alison,2010-09-17,0.097282
1,Alison,2010-09-18,0.260619
2,Alison,2010-09-19,0.205049
3,Alison,2010-09-20,0.256245
4,Alison,2010-09-21,0.243576
...,...,...,...
108536,Zelfa,2016-06-12,0.123130
108537,Zelfa,2016-06-13,0.094405
108538,Zelfa,2016-06-14,0.070772
108539,Zelfa,2016-06-15,0.171706


Based on the data, these tortoises usually move more than 0.2 km a day. So I set the scale at 0.05 km, which is 25% of 0.2 km. If a tortoise moves less than 0.5 km in a day, it is considered it stops the migration on that date.

In [8]:
# get all rows of Alison
alison_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Alison']
alison_distance = alison_distance[alison_distance['distance_km'] < 0.05]
alison_distance

,individual-local-identifier,date,distance_km
865,Alison,2013-01-29,0.047450
934,Alison,2013-04-08,0.000000
944,Alison,2013-04-18,0.032741
945,Alison,2013-04-19,0.041854
949,Alison,2013-04-23,0.049314
1209,Alison,2014-01-08,0.046733
1229,Alison,2014-01-28,0.011658
1233,Alison,2014-02-01,0.034556
1263,Alison,2014-03-03,0.006865
1275,Alison,2014-03-15,0.023418


For Alison, on the days above it moved less than 0.05 km a day and on 2013-4-08 Alison did not move at all.
Therefore, these days are considered the days Alison stopped the migration. On the day after each of these dates, it started the migration again.

In [9]:
# get all rows of Marilyn
marilyn_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Marilyn']
marilyn_distance = marilyn_distance[marilyn_distance['distance_km'] < 0.05]
marilyn_distance

,individual-local-identifier,date,distance_km
62486,Marilyn,2015-07-19,0.027160
63368,Marilyn,2018-10-01,0.036059


For Marilyn, it stopped the migration on 2015-07-19 and 2018-10-01. It started the migration on 2015-07-20 and 2018-10-02.

In [10]:
# get all rows of Kitty
kitty_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Kitty']
kitty_distance = kitty_distance[kitty_distance['distance_km'] < 0.05]
kitty_distance

,individual-local-identifier,date,distance_km
51050,Kitty,2011-02-28,0.016876
51064,Kitty,2011-03-14,0.018676
51065,Kitty,2011-03-15,0.033282
51067,Kitty,2011-03-17,0.031665
51068,Kitty,2011-03-18,0.043269
51420,Kitty,2012-03-04,0.036523
51421,Kitty,2012-03-05,0.037915
52691,Kitty,2015-08-27,0.048172
53220,Kitty,2017-04-16,0.039018
53377,Kitty,2017-09-20,0.024332


For Kitty, on the days above it moved less than 0.05 km a day.
Therefore, these days are considered the days Kitty stopped the migration. On the day after each of these dates, it started the migration again.

In [11]:
# get all rows of Connor
connor_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Connor']
connor_distance = connor_distance[connor_distance['distance_km'] < 0.05]
connor_distance

,individual-local-identifier,date,distance_km


The data shows us that connor moved more than 0.5 km every day, so we need to re-filter the data.

In [12]:
connor_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Connor']
connor_distance = connor_distance[connor_distance['distance_km'] < 0.06]
connor_distance

,individual-local-identifier,date,distance_km
16813,Connor,2013-09-22,0.059397


In [13]:
connor_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Connor']
connor_distance = connor_distance[connor_distance['distance_km'] < 0.07]
connor_distance

,individual-local-identifier,date,distance_km
16550,Connor,2013-01-02,0.065562
16813,Connor,2013-09-22,0.059397


By increasing the distance to 0.06 and 0.07 km, we finally get some dates.
Then we set 0.07 km as the limit value for Connor so it stopped the migration on 2013-01-02 and 2013-09-22.
Then we know it started migration on 2013-01-03 and 2013-09-23.

In [14]:
# Get all rows of Helber
helber_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Helber']
helber_distance = helber_distance[helber_distance['distance_km'] < 0.05]
helber_distance

,individual-local-identifier,date,distance_km
35287,Helber,2010-11-06,0.044835
35364,Helber,2011-01-22,0.045136


Helber stopped the migration on 2010-11-06 and 2011-01-22 and started the migration on 2010-11-07 and 2011-01-23.

In [15]:
# Get all rows of Christian
christian_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Christian']
christian_distance = christian_distance[christian_distance['distance_km'] < 0.05]
christian_distance

,individual-local-identifier,date,distance_km
12924,Christian,2010-11-24,0.041755
12928,Christian,2010-11-28,0.046556
13108,Christian,2011-05-27,0.046591
13138,Christian,2011-06-26,0.046569
13141,Christian,2011-06-29,0.037285
13151,Christian,2011-07-09,0.045392
13156,Christian,2011-07-14,0.044936
13158,Christian,2011-07-16,0.038552
13161,Christian,2011-07-19,0.039638
13167,Christian,2011-07-25,0.049519


According to the filtered data, Christian stopped pretty often. 
Dates above are considered the days Christian stopped the migration. On the day after each of these dates, it started the migration again.

In [16]:
# Get all rows of Steve Devine
sd_distance = daily_distance[daily_distance['individual-local-identifier'] == 'Steve Devine']
sd_distance = sd_distance[sd_distance['distance_km'] < 0.05]
sd_distance

,individual-local-identifier,date,distance_km
96135,Steve Devine,2010-09-20,0.032995
97960,Steve Devine,2015-09-30,0.032987
98160,Steve Devine,2016-05-10,0.046900
98968,Steve Devine,2018-07-27,0.005861


On these days Steve Devine moved less than 0.05 km.
Hence, it stopped the migration on 2010-09-20, 2015-09-30, 2016-05-10, and 2018-07-27.
It started the migration on 2010-09-21, 2015-10-01, 2016-05-11, and 2018-07-28.